### DP问题

**1. 分析子问题是否重复**

**2. 子问题存储**

**3. solution方法**

In [125]:
from collections import defaultdict

origin_price= [1,5,8,9,10,17,17,20,24,30]
price = defaultdict(int)
for i,p in enumerate(origin_price):
    price[i+1] = p
assert price[1] == 1

![](Picture1.png)

In [2]:
def r(n):
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)] # 这是个list，左右两边做比较取最大
    )

In [22]:
r(12)

48

In [126]:
solution = {}

In [127]:
@memo
def r(n):
    max_price, split_point = max(
        [(price[n],0)] + [(r(i) + r(n-i), i) for i in range(1,n)], key= lambda x:x[0] # max函数，里面有两个参数的比较，需要key指定如何比较
    )
    solution[n] = (split_point, n-split_point)
    return max_price

In [128]:
r(12)

35

In [129]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10)}

In [17]:
from functools import wraps

In [130]:
def memo(func):
    cache = {}
    @wraps(func)
    def _wrap(n):
        if n in cache: result = cache[n]
        else:
            result = func(n)
            cache[n] = result
        return result
    return _wrap

In [131]:
r(231)

691

In [139]:
from functools import lru_cache
@lru_cache(maxsize=2**10)
def r_new(n):
    max_price, split_point = max(
        [(price[n],0)] + [(r(i) + r(n-i), i) for i in range(1,n)], key= lambda x:x[0] # max函数，里面有两个参数的比较，需要key指定如何比较
    )
    solution[n] = (split_point, n-split_point)
    return max_price

In [140]:
r_new(999)

2995

In [136]:
def not_cut(split): return split == 0
def parse_solution(target_length, revenue_solution):
    left, right = revenue_solution[target_length]
    if not_cut(left): return [right]
    return parse_solution(left, revenue_solution) + parse_solution(right, revenue_solution)

In [137]:
parse_solution(87, solution)

[1, 6, 10, 10, 10, 10, 10, 10, 10, 10]

### 装饰器

In [28]:
import time

In [76]:
def func_1(n):
    for i in range(n):
        print(n)

In [30]:
def func_slow(n):
    for i in range(n):
        time.sleep(0.2)
        print(n)

方法一

In [26]:
def call_time(func_1, arg): # 脚手架程序 ,把函数和函数的参数都传了进去（面向函数：作为参数，能够赋值，能够return）
    start = time.time()
    func_1(arg)
    print('used time: {}'.format(time.time() - start))

In [29]:
call_time(func_1, 10)

10
10
10
10
10
10
10
10
10
10
used time: 0.0010030269622802734


方法二

In [77]:
def get_call_time(func):
    def _wrap(arg):
        start = time.time()
        result = func(arg)
        print("used time:{}".format(time.time()-start))
        return result
    return _wrap

In [78]:
func_1_could_get_time = get_call_time(func_1)

In [79]:
func_1_could_get_time(10)

10
10
10
10
10
10
10
10
10
10
used time:0.0009975433349609375


方法三

In [71]:
# 加@不加@的区别就是下面最后一句话，加了@是将func_1作为参数传入get_call_time函数，这个方法的作用和上面的func_1_could_get_time一样
@get_call_time
def func_1(n):
    for i in range(n):
        print(n)

In [73]:
func_1(10)

10
10
10
10
10
10
10
10
10
10
used time:0.0009987354278564453


方法四

In [121]:
function_called_time = {}
def get_call_time2(func):
#     加这个会让下面的help描写的是内部函数的注释，不加会变成内部get_call_time的注释
#     @wraps(func) 
    def _inner(arg): ## *args, **kwargs
        """It's inner function"""
        global function_called_time
        fun_name = func.__name__
        function_called_time[fun_name] += 1
        result = func(arg)
        print("函数调用次数：{}".format(function_called_time[func.__name__]))
        return result
    return _inner

In [123]:
@get_call_time2
def func_1_new(n):
    """
    @param n: is the number of customers
    @return int: the customers value point
    """
    for i in range(n):
        print(n)
    return 0

In [119]:
help(func_1_new)

Help on function _inner in module __main__:

_inner(arg)
    It's inner function

